In [1]:
import sys, os

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_holes, remove_small_objects
from skimage.measure import regionprops, label, find_contours
import pandas

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from registration_utilities import *
from annotation_utilities import *
from learning_utilities import *
from cell_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
No vtk
Child returned 0


Setting environment for AWS compute node
rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.37 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.34 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.33 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


In [2]:
stack = 'MD593'
img_w, img_h = metadata_cache['image_shape'][stack]
first_sec, last_sec = metadata_cache['section_limits'][stack]

download_from_s3(DataManager.get_image_dir(stack=stack, version='cropped_gray'), is_dir=True)

In [3]:
patch_size = 500
stride = 500

half_size = patch_size/2

min_blob_area = 30
max_blob_area = 3000

region1_h = patch_size
region1_w = patch_size

grid_index_to_coordinate = grid_parameters_to_sample_locations(patch_size=patch_size, stride=stride, 
                                                               w=img_w, h=img_h)

In [4]:
def detect_blobs_otsu(sec):
    
    if is_invalid(stack=stack, sec=sec):
        return

    img = imread(DataManager.get_image_filepath(stack=stack, section=sec, resol='lossless', version='cropped_gray'))
    rgb_img = gray2rgb(img)
    
    tb_mask = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec)

    otsu_t_by_grid_index = {}
    binary_image = np.zeros_like(img, np.bool)

    for gi, (x, y) in enumerate(grid_index_to_coordinate):

        region1_y = y - half_size
        region1_x = x - half_size

        region_img = img[region1_y:region1_y+region1_h, region1_x:region1_x+region1_w]

        if region1_x + region1_w >= img_w or region1_y + region1_h >= img_h:
            continue

        tb_region1_xmin = region1_x / 32
        tb_region1_xmax = (region1_x + region1_w) / 32
        tb_region1_ymin = region1_y / 32
        tb_region1_ymax = (region1_y + region1_h) / 32

        if np.count_nonzero(np.r_[tb_mask[tb_region1_ymin, tb_region1_xmin],
            tb_mask[tb_region1_ymin, tb_region1_xmax],
            tb_mask[tb_region1_ymax, tb_region1_xmin],
            tb_mask[tb_region1_ymax, tb_region1_xmax]]) == 0:
            continue

        otsu_t = threshold_otsu(region_img)    
        otsu_t_by_grid_index[gi] = otsu_t

        if otsu_t < 130 or otsu_t > 200:
            continue

        binary_img = region_img < otsu_t
    #     binary_img = remove_small_objects(binary_img, min_size=min_blob_area, connectivity=2)
        assert np.any(binary_img)
        binary_img = remove_small_holes(binary_img, min_size=min_blob_area, connectivity=2)
        binary_image[region1_y:region1_y+region1_h, region1_x:region1_x+region1_w] = binary_img.copy()

    labelmap = label(binary_image)

#     blob_props = regionprops(labelmap)
#     for blob_prop in blob_props:
#         if blob_prop.area < min_blob_area or blob_prop.area > max_blob_area:
#             binary_image[blob_prop.coords[:,0], blob_prop.coords[:,1]] = 0

#     labelmap = label(binary_image)

    fp = get_cell_data_filepath('labelmap_otsu', ext='bp', stack=stack, sec=sec)
    create_parent_dir_if_not_exists(fp)
    bp.pack_ndarray_file(labelmap, fp)
    upload_to_s3(fp)

    fp = get_cell_data_filepath('otsu_t_by_grid_index', ext='hdf', stack=stack, sec=sec)
    save_hdf_v2(otsu_t_by_grid_index, fp)
    upload_to_s3(fp)

In [5]:
for sec in range(first_sec, last_sec+1)[100:101]:
    t = time.time()
    detect_blobs_otsu(sec)
    sys.stderr.write("detect_blobs_otsu: %.2f\n" % (time.time() - t)) 

/usr/local/lib/python2.7/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "


aws s3 cp /shared/CSHL_cells_v2/detected_cells/MD593/MD593-N34-2015.08.21-18.31.01_MD593_2_0101/MD593-N34-2015.08.21-18.31.01_MD593_2_0101_labelmap_otsu.bp s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD593/MD593-N34-2015.08.21-18.31.01_MD593_2_0101/MD593-N34-2015.08.21-18.31.01_MD593_2_0101_labelmap_otsu.bp


Child returned 0
0.81 seconds.


aws s3 cp /shared/CSHL_cells_v2/detected_cells/MD593/MD593-N34-2015.08.21-18.31.01_MD593_2_0101/MD593-N34-2015.08.21-18.31.01_MD593_2_0101_otsu_t_by_grid_index.hdf s3://mousebrainatlas-data/CSHL_cells_v2/detected_cells/MD593/MD593-N34-2015.08.21-18.31.01_MD593_2_0101/MD593-N34-2015.08.21-18.31.01_MD593_2_0101_otsu_t_by_grid_index.hdf


Child returned 0
0.38 seconds.
detect_blobs_otsu: 12.51


# See some examples

In [ ]:
plt.boxplot(otsu_t_by_grid_index.values());

In [ ]:
for _ in range(10):
    while True:
        region1_x = np.random.randint(0, img_w-1000, 1)[0]
        region1_y = np.random.randint(0, img_h-1000, 1)[0]

        tb_region1_xmin = region1_x / 32
        tb_region1_xmax = (region1_x + region1_w) / 32
        tb_region1_ymin = region1_y / 32
        tb_region1_ymax = (region1_y + region1_h) / 32

        if np.count_nonzero(np.r_[tb_mask[tb_region1_ymin, tb_region1_xmin],
        tb_mask[tb_region1_ymin, tb_region1_xmax],
        tb_mask[tb_region1_ymax, tb_region1_xmin],
        tb_mask[tb_region1_ymax, tb_region1_xmax]]) >= 3:
            break
            
    viz = visualize_blob_contour(binary_image[region1_y:region1_y+region1_h, region1_x:region1_x+region1_w], 
                                 rgb_img=rgb_img[region1_y:region1_y+region1_h, region1_x:region1_x+region1_w])
    
    plt.figure(figsize=(10,10));
    plt.imshow(viz)
    plt.show();